21.03.03 Python 데이터베이스 연동

# 10장. 데이터베이스연동 

# 1절. SQLite 데이터 베이스 연결

## 1.1 SQLite와 파이썬
- SQLite 데이터 베이스는 별도의 서버 프로세스 없이 SQL을 사용하여 DB 엑세스를 할 수 있도록 만든 **간단한 디스크 기반 데이터 베이스**를 제공하는 C 라이브러리
- SQLite는 프로토타입을 만들때 사용.
- 정식제품은 오라클 및 DMBS를 사용.
- DB browser 사용

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.33.0'

# 1.2 데이터 베이스 연결

In [5]:
# DB 연결 객체 생성시 파일이 ㅇ벗으면 생성, 있으며 그 파일과 연동.
conn = sqlite3.connect('data/ch10_example.db')
conn 

In [6]:
# 커서 객체 반환. 커서는 SQL문을 실행 시키고 결과 데이터를 조회하는데 사용
cursor = conn.cursor()

In [9]:
cursor.execute('''
    CREATE TABLE MEMBER (
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [11]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동',20,'hong@kildong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('김길동',40,'kim@kildong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('신길동',60,'shin@kildong.com')")

In [12]:
conn.commit() # conn.rollback() - 롤백

In [13]:
cursor.execute("SELECT * FROM MEMBER") # 실행결과는 cursor 가르키고 있음.

In [14]:
# select문의 결과를 받는 함수 : 
# fetchone() : 결과를 한행씩 받을 때 
# fetchmany(n) : 결과를 n개 받을 때 
# fetall(n) : 결과를 모두 받을 때
print(cursor.fetchall())

[('홍길동', 20, 'hong@kildong.com'), ('김길동', 40, 'kim@kildong.com'), ('신길동', 60, 'shin@kildong.com')]


In [15]:
print(cursor.fetchall()) # 한번 호출된 fetall()은 다시 호출시 결과값이 없다.

[]


In [16]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()
for member in members:
    print(member)

('홍길동', 20, 'hong@kildong.com')
('김길동', 40, 'kim@kildong.com')
('신길동', 60, 'shin@kildong.com')


In [18]:
members[0]

('홍길동', 20, 'hong@kildong.com')

In [24]:
cursor.execute("SELECT * FROM MEMBER")

while True:
    member = cursor.fetchone()
    if member is None:
        break
    print(member)
    

('홍길동', 20, 'hong@kildong.com')
('김길동', 40, 'kim@kildong.com')
('신길동', 60, 'shin@kildong.com')


In [26]:
cursor.execute("SELECT * FROM MEMBER")

for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'hong@kildong.com')
('김길동', 40, 'kim@kildong.com')


In [27]:
cursor.fetchmany(2)

[('신길동', 60, 'shin@kildong.com')]

In [28]:
cursor.close() # 생략가능
conn.close() # 꼭 close 해야 connection 종료

## 1.3 SQL 구문에 파라미터 사용하기
- qmark, named(추천) 방법

In [32]:
# 파라미터 사용하기 : qmark 방법 이용
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE name IN (?,?)",('홍길동','김철수'))
print(cursor.fetchall())

[('홍길동', 20, 'hong@kildong.com')]


In [33]:
name1 = input('검색할 이름1 : ')
name2 = input('검색할 이름2 : ')
names = (name1, name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
print(cursor.fetchall())

검색할 이름1 : 신길동
검색할 이름2 : 홍길동
[('홍길동', 20, 'hong@kildong.com'), ('신길동', 60, 'shin@kildong.com')]


In [37]:
# 파라미터 사용하기 : named 방법
name = input('검색 하고자 하는 이름 : ')
cursor.execute("SELECT * FROM MEMBER WHERE NAME*:name", {'name':name})
members = cursor.fetchall()
if len(members) == 0:
    print('해당 이름이 없습니다.')
else:
    print(cursor.fetchall())

검색 하고자 하는 이름 : 홍길동
해당 이름이 없습니다.


In [38]:
try:
    name = input('입력할 이름?')
    age = int('입력할 나이?')
    email = input('입력할 메일은?')
except:
    print('나이를 잘못 입력 하셨습니다. 초기 값으로 20살 입력')
    age = 20
    email = ('입력할 메일은 ?')
cursor.execute('INSERT INTO MEMBER VALUES(:name, :age, :email)', 
              {'name':name, 'age':age, 'email':email})

입력할 이름?박길동
나이를 잘못 입력 하셨습니다. 초기 값으로 20살 입력


In [39]:
cursor.execute('SELECT * FROM MEMBER')
print(cursor.fetchall())

[('홍길동', 20, 'hong@kildong.com'), ('김길동', 40, 'kim@kildong.com'), ('신길동', 60, 'shin@kildong.com'), ('박길동', 20, '입력할 메일은 ?')]


In [64]:
cursor.close()
conn.close()

ProgrammingError: Cannot operate on a closed database.

# 2절. Oracle DB 연결

In [2]:
# 데이터 베이스 설정(DBMS가 있는 서버 주소, 포트번호, sid 들을 설정)
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [60]:
conn = cx_Oracle.connect(dsn=oracle_dsn, user='scott', password='tiger')
conn

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [46]:
conn.close()

In [74]:
# 1. conn 얻어오는 방법 (1)
oracle_dsn = cx_Oracle.makedsn(host='localhost', port=1521, sid='xe')
conn = cx_Oracle.connect(dsn=oracle_dsn, user='scott', password='tiger')

In [56]:
conn.close()

In [3]:
# 2. conn 얻어오는 방법 (2)
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [6]:
# cussor 얻어오기
cursor = conn.cursor()

In [7]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [8]:
sql = "SELECT * FROM EMP WHERE DEPTNO=:deptno"
deptno = input('검색하고자 하는 부서 번호는 ?')
cursor.execute(sql, {'deptno':deptno})

for emp in cursor:
    print(emp)

검색하고자 하는 부서 번호는 ?10
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7934, 'MILLER', 'CLERK', 7782, datetime.datetime(1982, 1, 23, 0, 0), 1300.0, None, 10)


In [9]:
sql = "SELECT * FROM EMP WHERE DEPTNO=:deptno"
deptno = input('검색하고자 하는 부서 번호는 ?')
cursor.execute(sql, {'deptno':deptno})

emp = cursor.fetchall()
if len(emp)==0 :
    print('입력하신 부서번호의 데이터가 없습니다.')
else:
    for e in emp:
        print(e)

검색하고자 하는 부서 번호는 ?20
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7902, 'FORD', 'ANALYST', 7566, datetime.datetime(1981, 12, 3, 0, 0), 3000.0, None, 20)


In [76]:
import pandas as pd
cursor.execute("SELECT * FROM EMP")
data = cursor.fetchall() # 튜플 리스트
data_df = pd.DataFrame(data) # 데이터 프레임
data_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [77]:
# 각 필드의 특정 정보(필드명, display_size, 내부크기, 정확도, scale, null)
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [79]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [81]:
data_df.columns = [row[0] for row in cursor.description]

In [82]:
data_df.columns

Index(['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO'], dtype='object')

In [83]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [84]:
data_df.tail()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30
10,7876,ADAMS,CLERK,7788.0,1983-01-12,1100.0,NaN,20
11,7900,JAMES,CLERK,7698.0,1981-12-03,950.0,NaN,30
12,7902,FORD,ANALYST,7566.0,1981-12-03,3000.0,NaN,20
13,7934,MILLER,CLERK,7782.0,1982-01-23,1300.0,NaN,10


In [10]:
cursor.close()
conn.close()

# 3절. Maria DB 연결

In [87]:
import pymysql

In [89]:
conn = pymysql.connect(host="localhost",
                      port=3306,
                      db="kimdb",
                      user="root", passwd="mysql",
                      charset="utf8", autocommit=True)
conn

In [91]:
cursor = conn.cursor()
sql = "SELECT * FROM PERSONAL"
cursor.execute(sql)
result = cursor.fetchall()
for r in result:
    print(r)

(1001, 'bill', 'president', None, datetime.date(1989, 1, 10), 7000, None, 10)
(1111, 'smith', 'manager', 1001, datetime.date(1990, 12, 17), 1000, None, 10)
(1112, 'ally', 'salesman', 1116, datetime.date(1991, 2, 20), 1600, 500, 30)
(1113, 'word', 'salesman', 1116, datetime.date(1992, 2, 24), 1450, 300, 30)
(1114, 'james', 'manager', 1001, datetime.date(1990, 4, 12), 3975, None, 20)
(1116, 'johnson', 'manager', 1001, datetime.date(1991, 5, 1), 3550, None, 30)
(1118, 'martin', 'analyst', 1111, datetime.date(1991, 9, 9), 3450, None, 10)
(1121, 'kim', 'clerk', 1114, datetime.date(1990, 12, 8), 4000, None, 20)
(1123, 'lee', 'salesman', 1116, datetime.date(1991, 9, 23), 1200, 0, 30)
(1226, 'park', 'analyst', 1111, datetime.date(1990, 1, 3), 2500, None, 10)


In [92]:
personal = pd.DataFrame(result)
personal.columns = [row[0] for row in cursor.description]
personal

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20
5,1116,johnson,manager,1001.0,1991-05-01,3550,NaN,30
6,1118,martin,analyst,1111.0,1991-09-09,3450,NaN,10
7,1121,kim,clerk,1114.0,1990-12-08,4000,NaN,20
8,1123,lee,salesman,1116.0,1991-09-23,1200,0.0,30
9,1226,park,analyst,1111.0,1990-01-03,2500,NaN,10


In [93]:
cursor.close()
conn.close()